In [0]:
CREATE LIVE TABLE silver_streamers
AS
SELECT
  user_id,
  LOWER(username) AS username,
  display_name,
  user_type AS type,
  broadcaster_type,
  description,
  profile_image_url,
  offline_image_url,
  CAST(account_created_at AS TIMESTAMP) AS account_created_at,
  followers_count,
  broadcaster_language,
  current_game_id AS game_id,
  current_game_name AS game_name,
  current_title AS title,
  stream_delay AS delay,
  data_collection_time,
  collection_date,
  ROUND(DATEDIFF(current_timestamp(), CAST(account_created_at AS TIMESTAMP)) / 365.0, 2) AS account_age_years,
  CASE WHEN broadcaster_type = 'affiliate' THEN true ELSE false END AS is_affiliate,
  CASE WHEN broadcaster_type = 'partner' THEN true ELSE false END AS is_partner
FROM LIVE.bronze_streamers;

In [0]:
CREATE LIVE TABLE silver_streams
AS
SELECT
  id,
  user_id,
  LOWER(user_login) AS username,
  user_name,
  game_id,
  game_name,
  type,
  title,
  viewer_count,
  CAST(started_at AS TIMESTAMP) AS stream_start_time,
  language,
  thumbnail_url,
  tag_ids,
  tags,
  is_mature,
  ingestion_time,
  ingestion_date,
  ROUND((unix_timestamp(ingestion_time) - unix_timestamp(CAST(started_at AS TIMESTAMP))) / 60.0, 2) AS stream_duration_minutes,
  CASE
    WHEN ROUND((unix_timestamp(ingestion_time) - unix_timestamp(CAST(started_at AS TIMESTAMP))) / 60.0, 2) > 0
    THEN viewer_count / ROUND((unix_timestamp(ingestion_time) - unix_timestamp(CAST(started_at AS TIMESTAMP))) / 60.0, 2)
    ELSE NULL
  END AS viewers_per_minute
FROM LIVE.bronze_streams;

In [0]:
CREATE LIVE TABLE silver_joined_data
AS
SELECT
  s.user_id,
  s.username,
  s.display_name,
  s.account_created_at,
  s.account_age_years,
  s.followers_count,
  st.viewer_count,
  st.stream_duration_minutes,
  st.viewers_per_minute,
  st.title,
  st.game_name,
  st.language,
  st.is_mature,
  st.ingestion_time
FROM LIVE.silver_streamers s
JOIN LIVE.silver_streams st ON s.user_id = st.user_id;